In [1]:
from os import walk
from joblib import Parallel, delayed
import librosa
import numpy as np
import multiprocessing
import json
from deep_audio import Directory, Audio, JSON

In [2]:
num_cores = multiprocessing.cpu_count()

sampling_rate = 16000

path = f'audios/{sampling_rate}'

f = Directory.filenames(path)

data = {
    "mapping": [],
    "lpcc": [],
    "labels": []
}

In [3]:
def process_directory(dir, index):
    signal, sr = Audio.read(f'{path}/{dir}', sr=sampling_rate, normalize=False)

    signal = np.array(signal)

    signal = signal[:len(signal) - len(signal) % (sr * 5)]

    segments = len(signal) // (sr * 5)

    m = {
        "lpcc": [],
        "labels": [index] * segments
    }

    for i in range(segments):
        start_sample = sr * i * 5
        finish_sample = start_sample + (sr * 5)

        sample = signal[start_sample:finish_sample]

        lpcc = Audio.lpcc(sample, rate=sr)

        lpcc = lpcc.T

        m['lpcc'].append(lpcc.tolist())

    print(f'{dir} -> segments: {segments}')
    return m

In [4]:
def object_lpcc_to_json(m):
    data['mapping'] = [file.replace('.wav', '') for file in f]

    for i in m:
        data['lpcc'].extend(i['lpcc'])
        data['labels'].extend(i['labels'])

    JSON.create_json_file(f'processed/lpcc/lpcc_{sampling_rate}.json', data)


In [ ]:
if __name__ == '__main__':
    m = Parallel(n_jobs=num_cores, verbose=len(f), temp_folder='./tmp/')(
        delayed(process_directory)(i, j) for j, i in enumerate(f) if i is not None)
    object_lpcc_to_json(m)